# Refactored code for
* Setting up and running Ollama in Kaggle
* Downloading THUIAR dataset
* Zero-Shot Prompt
* Use LLM to classify intent from an input 'question' dataset
* To configure your file/folder paths, LLM, dataset, start_index and end_index for each run, please update the config.py file

This notebook will also be used as the base to test any fixes to the LLM intent classification pipeline.
* 2025.05.26: Updated results output file from JSON to Pickle, to store list of dictionaries. 1 dictionary contains the results for each record. Lists of dictionaries can be downloaded from multiple notebooks, then concatenated for analysis
* 2025.05.30: Update prompt and bulletpts_intent.
  * Check if dataset contains 'oos' (out of scope) category
  * If dataset has no 'oos' (out of scope) category, turn 1 category into 'oos'. Use updated categories in bulletpts_intent. Also update prompt instructions on when to classify an example as 'oos'



In [1]:
# 1. create dirs if they do not exist
import os
os.makedirs('/kaggle/working/src', exist_ok=True)
os.makedirs('/kaggle/working/prediction', exist_ok=True)

In [2]:
%%writefile /kaggle/working/src/setup_ollama.py
import os
import subprocess
import time
from src.config import Config # absolute import

# 1. Install Ollama (if not already installed)
try:
    # Check if Ollama is already installed
    subprocess.run(["ollama", "--version"], capture_output=True, check=True)
    print("Ollama is already installed.")
except FileNotFoundError:
    print("Installing Ollama...")
    subprocess.run("curl -fsSL https://ollama.com/install.sh  | sh", shell=True, check=True)

# 2. Start Ollama server in the background
print("Starting Ollama server...")
process = subprocess.Popen("ollama serve", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Wait for the server to initialize
time.sleep(5)


# 3. Pull the model
model_name = Config.model_name
print(f"Pulling {model_name} model...")
subprocess.run(["ollama", "pull", model_name], check=True)

# 4. Install Python client
subprocess.run(["pip", "install", "ollama"], check=True)

print("Ollama setup complete!")

Writing /kaggle/working/src/setup_ollama.py


In [3]:
%%writefile requirements.txt
pandas
# numpy

Writing requirements.txt


In [4]:
%%writefile /kaggle/working/src/__init__.py
# folder for config

Writing /kaggle/working/src/__init__.py


In [5]:
%%writefile /kaggle/working/src/config.py
class Config:
    target_dir = '/kaggle/working/data' # data directory to clone into
    cloned_data_dir = target_dir + '/data'
    prediction_dir = target_dir + '/prediction'
    dataset_name = 'banking' # options: 'banking', 'stackoverflow', 'oos'
    model_name = 'llama3.2'
    start_index=0
    end_index=None # eg: 10 or None (use end_index=None to process the full dataset)
    log_every_n_examples= 100 # 2
    force_oos = True  # NEW: Add flag to force dataset to contain 'oos' class for the last class value (sorted alphabetically), if 'oos' class does not exist in the original dataset

Writing /kaggle/working/src/config.py


In [6]:
%%writefile download_dataset.py
from src.config import Config
import os
import subprocess
target_dir = Config.target_dir # data directory to clone into
cloned_data_dir = Config.cloned_data_dir

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# do not clone dataset repo if cloned data folder exists
if os.path.exists(cloned_data_dir):
    print("Dataset has already been downloaded. If this is incorrect, please delete the Adaptive-Decision-Boundary 'data' folder.")
else:
    # Clone the repository
    subprocess.run(["git",
                    "clone",
                    "https://github.com/thuiar/Adaptive-Decision-Boundary.git",
                    target_dir
                   ])

Writing download_dataset.py


In [7]:
%%writefile predict_class.py
from src.config import Config
import pandas as pd
import os
import ollama
import json
import pickle
import time

# Config.target_dir
# Config.cloned_data_dir'
# Config.dataset_name
# Config.model_name
# Config.start_index
# Config.end_index
# Config.log_every_n_examples


#######################
# load data
#######################
def load_data(data_dir):
    """Loads train, dev, and test datasets from a specified directory."""

    main_df = pd.DataFrame()
    for split in ['train', 'dev', 'test']:
        file_path = os.path.join(data_dir, f'{split}.tsv')
        if os.path.exists(file_path):
          try:
            df = pd.read_csv(file_path, sep='\t')
            df['dataset'] = os.path.basename(data_dir)
            df['split'] = split
            main_df = pd.concat([main_df, df], ignore_index=True)
          except pd.errors.ParserError as e:
            print(f"Error parsing {file_path}: {e}")
            # Handle the error appropriately, e.g., skip the file, log the error, etc.
        else:
            print(f"Warning: {split}.tsv not found in {data_dir}")
    return main_df

all_data = pd.DataFrame()

data_dir = os.path.join(Config.cloned_data_dir, Config.dataset_name)
if os.path.exists(data_dir):
  df = load_data(data_dir)
  print(f"Loaded dataset into dataframe: {Config.dataset_name}")
  print(f"Dimensions: {df.shape}")
  print(f"Col names: {df.columns}")
else:
  print(f"Warning: Directory {data_dir} not found.")
#######################



#######################
# unique intents
#######################
sorted_intent = list(sorted(df.label.unique()))
print(f"Original dataset intents: {sorted_intent}")


# Added on 2025.05.30
# force last category (sorted alphabetically) to be 'oos', if original dataset does not have the 'oos' category
if Config.force_oos and 'oos' not in sorted_intent:
    # Replace last category with oos
    prompt_categories = sorted_intent[:-1] + ['oos']
    print(f"Replaced last category '{sorted_intent[-1]}' with 'oos'")
else:
    prompt_categories = sorted_intent


# unique intents - from set to bullet points (to use in prompts)
# bulletpts_intent = "\n".join(f"- {category}" for category in set_intent)
bulletpts_intent = "\n".join(f"- {category}" for category in prompt_categories)


print("Prepared unique intents")
#######################



#######################
# Prompt
#######################
# prompt 2 with less information/compute, improve efficiency
def get_prompt(dataset_name, split, question, categories):
    
    prompt = f'''
You are an expert in understanding and identifying what users are asking you.

Your task is to analyze an input query from a user.
Then assign the most appropriate category to the query from a predefined list below:
{categories}

If you are unable to find the most appropriate category, please assign to the 'oos' (i.e. out of scope) category.

===============================

Question: {question}

===============================

Provide your final classification in **valid JSON format** with the following structure:
{{
  "category": "your_chosen_category_name",
  "confidence": 0.99
}}


Ensure the JSON has:
- Opening and closing curly braces
- Double quotes around keys and string values
- Confidence as a number (not a string), with maximum 2 decimal places

Do not include any explanations or extra text.
            '''
    return prompt



#######################


#######################
# Model on 1 Dataset
#######################
# Save a list of dictionaries 
# containing a dictionary for each record's
# - predicted category
# - confidence level and
# - original dataframe values

def predict_intent(model_name, df, categories, start_index=0, end_index=None, log_every_n_examples=100):
    start_time = time.time()
    results = []  # Store processed results
    
    # Slice DataFrame based on start/end indices
    if end_index is None:
        subset_df = df.iloc[start_index:]
    else:
        subset_df = df.iloc[start_index:end_index+1]
    
    total_rows = len(subset_df)
    
    for row in subset_df.itertuples():
        prompt = get_prompt(row.dataset, row.split, row.text, categories)
        if row.Index == 0:
            print("Example of how prompt looks, for the 1st example in this subset of data")
            print(prompt)
        
        
        try:
            response = ollama.chat(model=model_name, messages=[
                {'role': 'user', 'content': prompt}
            ])
            msg = response['message']['content']
            parsed = json.loads(msg)

            
            # Safely extract keys with defaults - resolve parsing error
            # maybe LLM did not output a particular key-value pair
            category = parsed.get('category', 'error')
            confidence = parsed.get('confidence', 0.0)
            parsed = {'category': category, 'confidence': confidence}
        except (json.JSONDecodeError, KeyError, Exception) as e:
            parsed = {'category': 'error', 'confidence': 0.0}
        
        # Combine original row data with predictions
        results.append({
            "Index": row.Index,
            "text": row.text,
            "label": row.label,
            "dataset": row.dataset,
            "split": row.split,
            "predicted": parsed['category'],
            "confidence": parsed['confidence']
        })
        
        # Log progress
        if row.Index % log_every_n_examples == 0:
            elapsed_time = time.time() - start_time
            
            avg_time_per_row = elapsed_time / (row.Index - start_index + 1)
            remaining_rows = total_rows - (row.Index - start_index + 1)
            eta = avg_time_per_row * remaining_rows
            
            print(f"Processed original df idx {row.Index} (subset row {row.Index - start_index}) | "
                  f"Elapsed: {elapsed_time:.2f}s | ETA: {eta:.2f}s")
    
    return results  # Return list of dictionaries
    

print(f"Starting intent classification using {Config.model_name}")
subset_results = predict_intent(Config.model_name, 
                                df, 
                                bulletpts_intent, 
                                start_index = Config.start_index, 
                                end_index = Config.end_index,
                                log_every_n_examples = Config.log_every_n_examples)



# update end_index for filename (if None is used for the end of the df)
# Get the last index of the DataFrame
last_index = df.index[-1] 
# Use last index if Config.end_index is None
end_index = Config.end_index if Config.end_index is not None else last_index


# 2025.05.23 changed from JSON to PKL
# because we are saving list of dictionaries
# Save to PKL
with open(f'results_{Config.model_name}_{Config.start_index}_{end_index}.pkl', 'wb') as f:
    pickle.dump(subset_results, f)
print("Completed intent classification")


#######################


Writing predict_class.py


In [8]:
%%writefile /kaggle/working/main.py
import subprocess
import sys


# 1. Install libraries from requirements.txt
print("Installing dependencies...")
subprocess.run([sys.executable, "-m", "pip", "install", "-r", "/kaggle/working/requirements.txt"], check=True)

# 2. Run setup_ollama.py
print("Starting Ollama setup...")
# subprocess.run(["python3", "/kaggle/working/src/setup_ollama.py"], check=True)
print("Starting Ollama setup...")
subprocess.run(
    ["python3", "-m", "src.setup_ollama"],  # Run as a module
    cwd="/kaggle/working",  # Set working directory to parent of 'src'
    check=True
)

# 3. Run download_dataset.py
print("Downloading dataset...")
subprocess.run(["python3", "/kaggle/working/download_dataset.py"], check=True)

# 4. Run predict_class.py
print("Running prediction script...")
subprocess.run(["python3", "/kaggle/working/predict_class.py"], check=True)

Writing /kaggle/working/main.py


# Model on subset of examples

In [9]:
!python3 /kaggle/working/main.py

Installing dependencies...
Starting Ollama setup...
Starting Ollama setup...
Installing Ollama...
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Starting Ollama server...
Pulling llama3.2 model...
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manife

# Sanity check folders

In [10]:
!cd /kaggle/working/ && ls -la

total 1908
drwxr-xr-x 5 root root    4096 May 30 04:35 .
drwxr-xr-x 8 root root    4096 May 30 02:15 ..
drwxr-xr-x 7 root root    4096 May 30 02:17 data
-rw-r--r-- 1 root root     695 May 30 02:15 download_dataset.py
-rw-r--r-- 1 root root     846 May 30 02:15 main.py
---------- 1 root root  329913 May 30 04:35 __notebook__.ipynb
-rw-r--r-- 1 root root    6911 May 30 02:15 predict_class.py
drwxr-xr-x 2 root root    4096 May 30 02:15 prediction
-rw-r--r-- 1 root root      15 May 30 02:15 requirements.txt
-rw-r--r-- 1 root root 1577618 May 30 04:35 results_llama3.2_0_13082.pkl
drwxr-xr-x 3 root root    4096 May 30 02:15 src


In [11]:
!cd /kaggle/working/src && ls -la

total 24
drwxr-xr-x 3 root root 4096 May 30 02:15 .
drwxr-xr-x 5 root root 4096 May 30 04:35 ..
-rw-r--r-- 1 root root  597 May 30 02:15 config.py
-rw-r--r-- 1 root root   20 May 30 02:15 __init__.py
drwxr-xr-x 2 root root 4096 May 30 02:15 __pycache__
-rw-r--r-- 1 root root  965 May 30 02:15 setup_ollama.py


In [12]:
!cd /kaggle/working/data/data && ls -la

total 20
drwxr-xr-x 5 root root 4096 May 30 02:17 .
drwxr-xr-x 7 root root 4096 May 30 02:17 ..
drwxr-xr-x 2 root root 4096 May 30 02:17 banking
drwxr-xr-x 2 root root 4096 May 30 02:17 oos
drwxr-xr-x 2 root root 4096 May 30 02:17 stackoverflow
